In [64]:
import pandas as pd
from math import floor
import itertools
import numpy as np
from functools import partial

def prime_factorization(n):
    """Return the prime factorization of `n`.

    Parameters
    ----------
    n : int
        The number for which the prime factorization should be computed.

    Returns
    -------
    dict[int, int]
        List of tuples containing the prime factors and multiplicities of `n`.

    """
    prime_factors = {}

    i = 2
    while i**2 <= n:
        if n % i:
            i += 1
        else:
            n /= i
            try:
                prime_factors[i] += 1
            except KeyError:
                prime_factors[i] = 1

    if n > 1:
        try:
            prime_factors[n] += 1
        except KeyError:
            prime_factors[n] = 1
    return prime_factors

def proc_SYSCLK(pll,div,crystal=10,min=40,max=90):
    SYSCLK= pd.concat([pd.Series({(d,p):crystal*p/d for p,d in itertools.product(pll,div)}).rename('SYSCLK')],axis=1)
    return SYSCLK[SYSCLK.SYSCLK>45][SYSCLK.SYSCLK<=90]

tick_ADS1220_turbo_mode = pd.DataFrame(
[[90            ,          45560,      45618   ],
 [180           ,          23064,      23122   ],
 [350           ,          11832,      11890   ],
 [660           ,          6232 ,      6290    ],
 [1200          ,          3448 ,      3506    ],
 [2000          ,          2072 ,      2130    ]],
 columns=["NOMINAL_DR"  ,          "CC" ,     "SC"     ]
).set_index("NOMINAL_DR")

tick_ADS1220_normal_mode = pd.DataFrame(
[[20  ,    204768,    204850 ],
[45  ,    91120 ,    91218  ],
[90  ,    46128 ,    46226  ],
[175 ,    23664 ,    23762  ],
[330 ,    12464 ,    12562  ],
[600 ,    6896  ,    6994   ],
[1000,    4144  ,    4242   ]],
columns=["NOMINAL_DR"  ,          "CC" ,     "SC"     ]
).set_index("NOMINAL_DR")
tick_ADS1220 = pd.concat({'normal_mode':tick_ADS1220_normal_mode,'turbo_mode':tick_ADS1220_turbo_mode})


## Come valutare l'errore nella frequenza

Ho due orologi aventi una frequenza $f$ e $\tilde f$  


- Dopo quanti tick $N1$ ho uno sfasamento pari a un secondo? Quanti secondi sono passati $T1$? 
$$N1 = \frac{1}{dt-\tilde{dt}} = \frac{f\tilde f}{\tilde f-f}$$
$$T1 = \frac{dt}{dt-\tilde{dt}} = \frac{\tilde f}{\tilde f-f}$$

- Dopo quanti tick $N2$ ho uno sfasamento pari a $dt$? Quanti secondi sono passati $T2$?

$$N2 = \frac{dt}{dt-\tilde{dt}} = \frac{\tilde f}{\tilde f-f}$$

$$T2 = \frac{dt^2}{dt-\tilde{dt}} = \frac{1}{f}\frac{\tilde f}{\tilde f-f}$$

con $a=1- \frac{f}{\tilde f}=\frac{\tilde f-f}{\tilde f}$ si ottiene:

$$N1 = \frac{f}{a} $$
$$N2 =T1 = \frac{1}{a}$$
$$T2 =\frac{1}{fa}$$



## Scelta DR
Obbiettivo é  trovare la DR date le Frequenze CPU che minimizzano l'errore  

In [65]:
DR=[200, 400,500,600,800,1000]+[2**n for n in range(8,11)]
DR

[200, 400, 500, 600, 800, 1000, 256, 512, 1024]

In [95]:
def get_near_freq(f,f_pwm):
    return f_pwm[np.abs(f_pwm-f).idxmin()]

def calc_err(r):
    f, f_pwm ,DR,CC = r['f'],r['f_pwm'],r.name[0],r['CC']
    a=1-f/f_pwm
    return pd.Series({'N1':f/a, 'N2_T1':1/a,'T2':np.abs(1/(DR*a)),'true_DR':f_pwm/CC})

def calc_freq(DR,crystal):
    SYSCLK= proc_SYSCLK(np.arange(19).astype(int),[1,2,4],crystal= crystal )
    print(SYSCLK)
    
    #
    f = pd.concat({dr:pd.concat(( tick_ADS1220[['CC']], tick_ADS1220['CC'].apply(lambda x: x*dr).rename('f')),axis=1) for dr in DR})
    f=f[(f.f>500000) & (f.f<=4500000)]
    #
    f_pwm = pd.concat({n: pd.concat([SYSCLK,SYSCLK.SYSCLK.apply(lambda x: x*1000000/n).rename('f_pwm')],axis=1) for n in range(10,90)})
    f_pwm=f_pwm[(f_pwm.f_pwm>500000) & (f_pwm.f_pwm<=4500000)].dropna()
    #f_pwm_r=np.array(list(set(f_pwm.values)))
    f['f_pwm']=f.f.map(partial(get_near_freq,f_pwm=f_pwm.f_pwm))

    res=pd.concat([f,f.apply(calc_err,axis=1)],axis=1)
    res = res.sort_values(by='T2',ascending=False)[:10]
    f_pwm=f_pwm[f_pwm.f_pwm.isin(res.f_pwm[:10].values)]
    return (res, f_pwm)



In [99]:
res, f_pwm=calc_freq(DR,5.1851)
res, f_pwm=calc_freq(DR,10)
res

/home/esr/anaconda3/envs/myp/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


       SYSCLK
1 9   46.6659
  10  51.8510
  11  57.0361
  12  62.2212
  13  67.4063
  14  72.5914
  15  77.7765
  16  82.9616
  17  88.1467
2 18  46.6659
      SYSCLK
1 5     50.0
  6     60.0
  7     70.0
  8     80.0
  9     90.0
2 10    50.0
  11    55.0
  12    60.0
  13    65.0
  14    70.0
  15    75.0
  16    80.0
  17    85.0
  18    90.0


CC        f         f_pwm            N1  \
                NOMINAL_DR                                              
200 turbo_mode  1200        3448   689600  6.896552e+05  8.620000e+09   
    normal_mode 600         6896  1379200  1.379310e+06  1.724000e+10   
400 turbo_mode  1200        3448  1379200  1.379310e+06  1.724000e+10   
    normal_mode 600         6896  2758400  2.758621e+06  3.448000e+10   
500 turbo_mode  1200        3448  1724000  1.724138e+06  2.155000e+10   
600 turbo_mode  1200        3448  2068800  2.068966e+06  2.586000e+10   
256 turbo_mode  660         6232  1595392  1.595745e+06  7.218533e+09   
    normal_mode 1000        4144  1060864  1.060606e+06 -4.362105e+09   
800 turbo_mode  1200        3448  2758400  2.758621e+06  3.448000e+10   
256 normal_mode 600         6896  1765376  1.764706e+06 -4.648989e+09   

                                   N2_T1         T2     true_DR  
                NOMINAL_DR                                       
200 turbo_mode  1200        12500.000000  62.500000  200.016001  
    normal_mode 600         12500.000000  62.500000  200.016001  
400 turbo_mode  1200        12500.000000  31.250000  400.032003  
    normal_mode 600         12500.000000  31.250000  400.032003  
500 turbo_mode  1200        12500.000000  25.000000  500.040003  
600 turbo_mode  1200        12500.000000  20.833333  600.048004  
256 turbo_mode  660          4524.613900  17.674273  256.056592  
    normal_mode 1000        -4111.842105  16.061883  255.937756  
800 turbo_mode  1200        12500.000000  15.625000  800.064005  
256 normal_mode 600         -2633.426966  10.286824  255.902825

In [100]:
f_pwm

SYSCLK         f_pwm
29 1 5     50.0  1.724138e+06
     6     60.0  2.068966e+06
     8     80.0  2.758621e+06
   2 10    50.0  1.724138e+06
     12    60.0  2.068966e+06
     16    80.0  2.758621e+06
34 1 6     60.0  1.764706e+06
   2 12    60.0  1.764706e+06
47 2 15    75.0  1.595745e+06
51 1 9     90.0  1.764706e+06
   2 18    90.0  1.764706e+06
58 1 8     80.0  1.379310e+06
   2 16    80.0  1.379310e+06
66 1 7     70.0  1.060606e+06
   2 14    70.0  1.060606e+06
87 1 6     60.0  6.896552e+05
   2 12    60.0  6.896552e+05

In [ ]:
crystal = [5.1851]
best={}
for c in crystal:
    best[c]=calc_freq(DR,c)

results=pd.concat(best)
results

In [26]:
results.loc[[results.T2.idxmax()]]

,,,,CC,f,f_pwm,N1,N2_T1,T2
,,,NOMINAL_DR,,,,,,
5.185,256,normal_mode,330,12464,3190784,3.190769e+06,-6.893423e+11,-216041.666668,843.91276
